<!-- Feature Embedding Process during registration -->

In [1]:
from tensorflow.keras.models import Model, load_model   
from tensorflow.keras.layers import Input, Conv2D, Dense, GlobalMaxPooling2D
from tensorflow.keras.applications import VGG16
from keras.saving import register_keras_serializable
import matplotlib.pyplot as plt
import cv2
import numpy as np
import tensorflow as tf

In [2]:
@register_keras_serializable()
class FaceDetector(Model):
    def __init__(self, faceDetector, **kwargs):
        super().__init__(**kwargs)
        self.model=faceDetector

    def compile(self, opt, classLoss, regressLoss,  **kwargs):
            # super().compile(**kwargs)
            # self.opt=opt
            # self.cLoss=classLoss
            # self.lLoss=regressLoss

            super().compile(
                  optimizer=opt,
                  loss=[classLoss,regressLoss],
                  **kwargs
            )
            self.opt=opt
            self.cLoss=classLoss
            self.lLoss=regressLoss

            # self.cLoss=loss.get('classification')
            # self.lLoss=loss.get('regression')
            # super().compile(optimizer=opt, loss={'classification': self.cLoss, 'regression': self.lLoss},**kwargs)
            # self.opt=opt

    def train_step(self, batch, **kwargs):
        X,y=batch
        # Ensure the shape of X is (batch_size, 120, 120, 3)
        X = tf.ensure_shape(X, [None, 120, 120, 3])

        y = list(y)

        # # Ensure y has the expected shape (batch_size, num_classes) for class labels
        y[0] = tf.ensure_shape(y[0], [None, 1])  # Adjust num_classes as needed
        y[1] = tf.ensure_shape(y[1], [None, 4])  # Adjust num_boxes as needed

        y = tuple(y)


        # print(f"Rank of X: {tf.rank(X)}")
        # print(f"Rank of y: {tf.rank(y)}")
        with tf.GradientTape()  as tape:
              classes,coordinates,features= self.model(X,training=True)
              classLoss=self.cLoss(y[0],classes)
              localizationLoss=self.lLoss(tf.cast(y[1],tf.float32),coordinates)
              totalLoss=0.5*classLoss+localizationLoss
              grad=tape.gradient(totalLoss,self.model.trainable_variables)

        opt.apply_gradients(zip(grad,self.model.trainable_variables))   
        return {'totalLoss':totalLoss, 'classLoss':classLoss, 'regressLoss':localizationLoss }

    def test_step(self, batch, **knwargs):
              
            X,y=batch

            # Ensure the shape of X is (batch_size, 120, 120, 3)
            X = tf.ensure_shape(X, [None, 120, 120, 3])

            y = list(y)

            # # Ensure y has the expected shape (batch_size, num_classes) for class labels
            y[0] = tf.ensure_shape(y[0], [None, 1])  # Adjust num_classes as needed
            y[1] = tf.ensure_shape(y[1], [None, 4])  # Adjust num_boxes as needed

            y = tuple(y)

            classes,coordinates= self.model(X,training=True)
            classLoss=self.cLoss(y[0],classes)
            localizationLoss=self.lLoss(tf.cast(y[1],tf.float32),coordinates)
            totalLoss=0.5*classLoss+localizationLoss

            return {'totalLoss':totalLoss, 'classLoss':classLoss, 'regressLoss':localizationLoss}
    
    def call(self, X, **kwargs):
          return self.model(X, **kwargs)
    
    def get_config(self):
        config = super().get_config()
        config.update({"faceDetector": self.model.to_json()})  
        return config

    @classmethod
    def from_config(cls, config):
        from tensorflow.keras.models import model_from_json
        faceDetector = config.pop("faceDetector") 
        faceDetector = model_from_json(faceDetector)
        return cls(faceDetector, **config)
            

In [3]:
model = load_model("../Model/modelFaceDetection.keras", custom_objects={"FaceDetector": FaceDetector}, compile=False)

In [10]:
cap=cv2.VideoCapture(0)


# emp_name=input("Name of Employee")
# emp_id=input("Id of Employee")

frames4=[]

frame_count = 0

while cap.isOpened():
    _,frame=cap.read()
    frame = frame[50:500, 50:500,:]
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    resized = tf.image.resize(rgb, (120,120))
    yPred=model.predict(np.expand_dims(resized/255,0))
    coords=yPred[1][0]

    frame_count=frame_count+1

    x,y,h,w=np.multiply(coords, [450,450,450,450]).astype(int)
    frameh=frame[y:y+h, x:x+w].copy()
    
    if yPred[0]>0.7:
        cv2.rectangle(frame, tuple(np.multiply(coords[:2], [450,450]).astype(int)), tuple(np.multiply(coords[2:], [450,450]).astype(int)), (255,200,0), 2)

        cv2.rectangle(frame, 
                      tuple(np.add(np.multiply(coords[:2], [450,450]).astype(int), 
                                    [0,-30])),
                      tuple(np.add(np.multiply(coords[:2], [450,450]).astype(int),
                                    [80,0])), 
                            (255,0,0), -1)
        
        cv2.putText(frame, 'face', tuple(np.add(np.multiply(coords[:2], [450,450]).astype(int),
                                               [0,-5])),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)

    cv2.imshow('Face Track', frame)

    if frame_count%8==0 and len(frames4)<4 : 
        frames4.append(frameh)

    if (cv2.waitKey(1) & 0xFF == ord('b')) or len(frames4)>=4:
        break

cap.release()
cv2.destroyAllWindows()

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 175ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 190ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 169ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 155ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 149ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 151ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 140ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 155ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 158ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 141ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 134ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 138ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 142ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 126ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 141ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 136ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 146ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 130ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 119ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 145ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 146ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 150ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 143ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 129ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 131ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 

In [12]:

for i, frame in enumerate(frames4):
    if frame is not None and frame.size > 0:
        cv2.imshow(f"Frame {i+1}", frame)
        cv2.waitKey(1000)  # Show each frame for 1 second (1000 ms)
        cv2.destroyAllWindows()  # Close previous window before next
    else:
        print(f"Frame {i+1} is empty or invalid")

cv2.destroyAllWindows()


In [9]:
len(frames4)

4

In [22]:
frames4[0].shape

(310, 198, 3)

In [13]:
def load_facenet():
    from tensorflow.keras.applications import EfficientNetB0
    from tensorflow.keras.layers import GlobalAveragePooling2D
    # model_url = "https://tfhub.dev/google/efficientnet/b0/feature-vector/1"
    # efficientnet = hub.KerasLayer(model_url, trainable=False)

    base_model = EfficientNetB0(weights='imagenet', include_top=False)
    base_model.trainable=False

    output_layer = GlobalAveragePooling2D()(base_model.output)

    dense_layer=Dense(units=128, activation=None, name="embedding")(output_layer)
    
    model = Model(inputs=base_model.input, outputs=dense_layer)
    return model


In [14]:
efficientnetModel=load_facenet()

In [ ]:
def preprocess_face(face):
    face=cv2.resize(face,(224,224))
    face=face.astype('float32')/255.0
    face=np.expand_dims(face,axis=0)
    return face

def extract_embedding(model,face):
    face=preprocess_face(face)
    embedding=model.predict(face)
    return embedding

def store_embeddings(frames4):
    framesEmbeddings=[]
    for face in frames4:
        face=extract_embedding(efficientnetModel, face)
        framesEmbeddings.append(face)
    return framesEmbeddings    

In [40]:
frameEmbeddings=store_embeddings( frames4)

print(frameEmbeddings)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
[array([[ 0.0849017 , -0.26695487,  0.45917496,  0.07800081, -0.21510078,
         0.5275463 ,  0.1663514 ,  0.50519717, -0.26687843,  0.34458774,
        -0.02882018,  0.14124547,  0.32989237,  0.13567883, -0.25143814,
         0.2346692 ,  0.2873069 ,  0.32980654,  0.21239583, -0.31991687,
         0.2971754 , -0.22426757, -0.48768744, -0.03604997,  0.05764559,
        -0.04620275, -0.06819285,  0.47016516,  0.18816702, -0.01578464,
        -0.38425925,  0.00970387,  0.05742314, -0.14721973,  0.09067041,
        -0.19697692,  0.78042585, -0.16341224,  0.42748412,  0.06623575,
        -0.05371849,  0.1518552 , -0.01437322, -0.2462003 ,  0.5326759 ,
         0.05843233, -0.18141302, -0.03058694,  0.29623637, -0.32162526,
         0.00497281,  0.2953813 , -0.13078633, -0.05492041, -0.39509258,
        -0.19305828,  0.6445426 , -0.1371155